# RAG and Semantic Retrieval on a Document Collection

Deep Search allows users to interact with the documents using conversational AI, i.e. you interact with a virtual assistant which answer your questions using the information in the corpus.

In this example we demonstrate how achive the same interaction programmatically.

### Access required

The content of this notebook requires access to Deep Search capabilities which are not
available on the public access system.

[Contact us](https://ds4sd.github.io) if you are interested in exploring
these Deep Search capabilities.


### GenAI Integration required

When interacting with the virtual assistant, Deep Search requires a connection to a Generative AI API. Currently, we support connections to [watsonx.ai](https://www.ibm.com/products/watsonx-ai) or the IBM-internal GenAI platform BAM.

Deep Search allows custom GenAI configurations for each project.
In the following example you will require to work in a project which has such GenAI capabilities activated.

### Set notebook parameters


In [4]:
from dsnotebooks.settings import CollQANotebookSettings

# notebooks settings auto-loaded from .env / env vars
notebook_settings = CollQANotebookSettings()

PROFILE_NAME = notebook_settings.profile      # the profile to use
PROJ_KEY = notebook_settings.proj_key         # the project to use
INDEX_KEY = notebook_settings.sem_on_idx_key  # the collection to use

SKIP_INGESTED_DOCS = notebook_settings.skip_ingested_docs  # whether to skip any already semantically ingested docs

RETR_K = notebook_settings.retr_k             # the number of search results to retrieve
TEXT_WEIGHT = notebook_settings.text_weight   # the weight of lexical search (0.0: semantic-only, 1.0: lexical-only, anything in between: hybrid search)
RERANK = notebook_settings.rerank             # whether to rerank the search results
RAISE = notebook_settings.raise_on_sem_err    # whether semantic operation errors should raise an exception or be reflected in response fields

### Import example dependencies

In [5]:
# Import standard dependenices
import pandas as pd
import rich

# IPython utilities
from IPython.display import display, Markdown

# Import the deepsearch-toolkit
from deepsearch.cps.client.api import CpsApi
from deepsearch.cps.client.components.elastic import ElasticProjectDataCollectionSource
from deepsearch.cps.queries import DataQuery, RAGQuery, SemanticQuery
from deepsearch.cps.queries.results import RAGResult, SearchResult, SearchResultItem


### Connect to Deep Search

In [6]:
api = CpsApi.from_env(profile_name=PROFILE_NAME)

### Utils

In [7]:
def render_provenance_url(
        api: CpsApi,
        coords: ElasticProjectDataCollectionSource,
        retr_item: SearchResultItem,
):
    ## compute URL to the document in the Deep Search UI
    item_index = int(retr_item.main_path[retr_item.main_path.rfind(".")+1:])
    doc_url = api.documents.generate_url(
        document_hash=retr_item.doc_hash,
        data_source=coords,
        item_index=item_index,
    )
    display(Markdown(f"The provenance of the answer can be inspected on the [source document]({doc_url})."))

---

Prepare the collection coordinates:

In [8]:
coll_coords = ElasticProjectDataCollectionSource(
    proj_key=PROJ_KEY,
    index_key=INDEX_KEY,
)

We are using a small collection, so we can just list its documents to get an idea of its contents (for more details on querying, check the [Data Query Quick Start](https://github.com/DS4SD/deepsearch-examples/tree/main/examples/data_query_quick_start)).

In [9]:
# Prepare the data query
query = DataQuery(
    search_query="*",  # The search query to be executed
    source=[           # Which fields of documents we want to fetch
            "file-info.document-hash",
            "file-info.filename",
            # "description.title",
    ],
    coordinates=coll_coords,  # The data collection to be queries
)

# Query Deep Search for the documents matching the query
results = []
query_results = api.queries.run(query)
for row in query_results.outputs["data_outputs"]:
        # Add row to results table
        results.append({
            "Filename": row["_source"]["file-info"]["filename"],
            "DocHash": row["_source"]["file-info"]["document-hash"],
            # "Title": row["_source"].get("description", {}).get("title"),
        })

print(f'Finished fetching all data. Total is {len(results)} records.')

# Visualize the table with all results
df = pd.json_normalize(results)
display(df)

Finished fetching all data. Total is 1 records.


,Filename,DocHash
0,2206-01062.pdf,5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c8...


## Prepare source

In [10]:
from deepsearch.cps.client.components.documents import PrivateDataCollectionSource, PrivateDataDocumentSource, PublicDataDocumentSource

data_source = PrivateDataCollectionSource(
    source=coll_coords,
)

## Ingestion

In the cell below we show how to semantically index your collection (indexing of already indexed docs is controlled via param `skip_ingested_docs`):

In [ ]:
# launch the ingestion of the collection for DocumentQA
task = api.documents.semantic_ingest(
    project=PROJ_KEY,
    data_source=data_source,
    skip_ingested_docs=SKIP_INGESTED_DOCS,
)

# wait for the ingestion task to finish
api.tasks.wait_for(PROJ_KEY, task.task_id)

## RAG

In [12]:
question = "Where is the IBM lab in Zurich?"

# submit natural-language query on collection
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=RETR_K,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

timing = f"overall={api_output.timings.overall:.2f}  "

for k, v in api_output.timings.tasks["QA"].details.items():
    timing += f"{k}={v:.3f}  "

rich.print(rag_result)

print(f"{timing.strip()}")

RAGResult(
    answers=[
        RAGAnswerItem(
            answer='The IBM lab in Zurich is in Rueschlikon, Switzerland.',
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
                        chunk='DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis\nMichele 
Dolfi IBM Research Rueschlikon, Switzerland dol@zurich.ibm.com',
                        main_path='main-text.4',
                        path_group=['main-text.1', 'main-text.4'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.1',
                    'main-text.2',
                    'main-text.3',
                    'main-text.4',
                    'main-text.5',
                    'main-text.6',
                    'main-text.7',
                    'main-text.8',
                    'main-text.9',
                    'main-text.10',
                    'main-text.11',
                    'main-text.12'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
            chunk='DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis\nMichele Dolfi IBM 
Research Rueschlikon, Switzerland dol@zurich.ibm.com',
            main_path='main-text.4',
            path_group=['main-text.1', 'main-text.4'],
            source_is_text=True
        )
    ]
)

overall=2.17  pipeline=1.531  encode=0.031  search=0.024  rerank=0.000  generate=1.259


Additionally, we can generate a provenance URL to the document in the Deep Search UI:

In [13]:
render_provenance_url(api=api, coords=coll_coords, retr_item=rag_result.answers[0].grounding.retr_items[0])

The provenance of the answer can be inspected on the [source document](https://cps.foc-deepsearch.zurich.ibm.com/projects/6db72b8a4b537e1e6fa42f4403eb602a8ef0318e/library/private/2a052da60ce6f7d69837c6c2a492124b17f0a1d6?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjIyYTA1MmRhNjBjZTZmN2Q2OTgzN2M2YzJhNDkyMTI0YjE3ZjBhMWQ2JTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyNWRmYmQ4YzExNWExNWZkMzM5NmI2ODQwOTEyNGNmZWUyOWZjOGVmYWM3YjVjODQ2NjM0ZmY5MjRlNjM1ZTBkYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMjVkZmJkOGMxMTVhMTVmZDMzOTZiNjg0MDkxMjRjZmVlMjlmYzhlZmFjN2I1Yzg0NjYzNGZmOTI0ZTYzNWUwZGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0E0JTdEJTdE).

Let us try out a different question on our document corpus.
Here we also include (commented out) various additional parameters the user can optionally set:
- `retr_k`: number of items to retrieve
- `text_weight`: weight of lexical search (`0.0`: fully semantic search, `1.0`: fully lexical search, anything in-between: hybrid search)
- `rerank`: whether to rerank the retrieval results
- `gen_ctx_extr_method` (Literal["window", "page"], optional): method for gen context extraction from document; defaults to "window"
- `gen_ctx_window_size` (int, optional): (relevant only if `gen_ctx_extr_method` is "window") max chars to use for extracted gen context (actual extraction quantized on doc item level); defaults to 5000
- `gen_ctx_window_lead_weight` (float, optional): (relevant only if `gen_ctx_extr_method` is "window") weight of leading text for distributing remaining window size after extracting the `main_path`; defaults to 0.5 (centered around `main_path`)
- `return_prompt` (bool, optional): whether to return the instantiated prompt; defaults to False
- `gen_timeout` (float, optional): timeout for LLM generation; defaults to None, i.e. determined by system

For more details refer to `deepsearch.cps.queries.RAGQuery`.

In [14]:
question = "Who came up with the term 'machine learning'?"

# submit natural-language query on collection
question_query = RAGQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional retrieval params
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    # rerank=RERANK,

    ## optional generation params
    # model_id="ibm-mistralai/mixtral-8x7b-instruct-v01-q",
    # gen_params={"random_seed": 42, "max_new_tokens": 1024},
    # prompt_template="Answer the query based on the context.\n\nContext: {{ context }}\n\nQuery: {{ query }}",

    # gen_ctx_extr_method="window",
    # gen_ctx_window_size=5000,
    # gen_ctx_window_lead_weight=0.5
    # return_prompt=True,
    # gen_timeout=10.0,
)
api_output = api.queries.run(question_query)
rag_result = RAGResult.from_api_output(api_output, raise_on_error=RAISE)

timing = f"overall={api_output.timings.overall:.2f}  "

for k, v in api_output.timings.tasks["QA"].details.items():
    timing += f"{k}={v:.3f}  "

rich.print(rag_result)

print(f"{timing.strip()}")

RAGResult(
    answers=[
        RAGAnswerItem(
            answer="The term 'machine learning' was introduced by Arthur Samuel in 1959 [Samuel1959].",
            grounding=RAGGroundingInfo(
                retr_items=[
                    SearchResultItem(
                        doc_hash='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
                        chunk='KEYWORDS\nPDF document conversion, layout segmentation, object-detection, data set, 
Machine Learning',
                        main_path='main-text.17',
                        path_group=['main-text.16', 'main-text.17'],
                        source_is_text=True
                    )
                ],
                gen_ctx_paths=[
                    'main-text.8',
                    'main-text.9',
                    'main-text.10',
                    'main-text.11',
                    'main-text.12',
                    'main-text.13',
                    'main-text.14',
                    'main-text.16',
                    'main-text.17',
                    'main-text.18',
                    'main-text.19',
                    'main-text.20',
                    'main-text.21',
                    'main-text.22'
                ]
            ),
            prompt=None
        )
    ],
    search_result_items=[
        SearchResultItem(
            doc_hash='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
            chunk='KEYWORDS\nPDF document conversion, layout segmentation, object-detection, data set, Machine 
Learning',
            main_path='main-text.17',
            path_group=['main-text.16', 'main-text.17'],
            source_is_text=True
        )
    ]
)

overall=2.12  pipeline=1.994  encode=0.037  search=0.037  rerank=0.000  generate=1.856


As seen by the returned `doc_hash`, this answer could came from a different document than the previous one.

In [15]:
render_provenance_url(api=api, coords=coll_coords, retr_item=rag_result.answers[0].grounding.retr_items[0])

The provenance of the answer can be inspected on the [source document](https://cps.foc-deepsearch.zurich.ibm.com/projects/6db72b8a4b537e1e6fa42f4403eb602a8ef0318e/library/private/2a052da60ce6f7d69837c6c2a492124b17f0a1d6?search=JTdCJTIycHJpdmF0ZUNvbGxlY3Rpb24lMjIlM0ElMjIyYTA1MmRhNjBjZTZmN2Q2OTgzN2M2YzJhNDkyMTI0YjE3ZjBhMWQ2JTIyJTJDJTIydHlwZSUyMiUzQSUyMkRvY3VtZW50JTIyJTJDJTIyZXhwcmVzc2lvbiUyMiUzQSUyMmZpbGUtaW5mby5kb2N1bWVudC1oYXNoJTNBJTIwJTVDJTIyNWRmYmQ4YzExNWExNWZkMzM5NmI2ODQwOTEyNGNmZWUyOWZjOGVmYWM3YjVjODQ2NjM0ZmY5MjRlNjM1ZTBkYyU1QyUyMiUyMiUyQyUyMmZpbHRlcnMlMjIlM0ElNUIlNUQlMkMlMjJzZWxlY3QlMjIlM0ElNUIlMjJfbmFtZSUyMiUyQyUyMmRlc2NyaXB0aW9uLmNvbGxlY3Rpb24lMjIlMkMlMjJwcm92JTIyJTJDJTIyZGVzY3JpcHRpb24udGl0bGUlMjIlMkMlMjJkZXNjcmlwdGlvbi5wdWJsaWNhdGlvbl9kYXRlJTIyJTJDJTIyZGVzY3JpcHRpb24udXJsX3JlZnMlMjIlNUQlMkMlMjJpdGVtSW5kZXglMjIlM0EwJTJDJTIycGFnZVNpemUlMjIlM0ExMCUyQyUyMnNlYXJjaEFmdGVySGlzdG9yeSUyMiUzQSU1QiU1RCUyQyUyMnZpZXdUeXBlJTIyJTNBJTIyc25pcHBldHMlMjIlMkMlMjJyZWNvcmRTZWxlY3Rpb24lMjIlM0ElN0IlMjJyZWNvcmQlMjIlM0ElN0IlMjJpZCUyMiUzQSUyMjVkZmJkOGMxMTVhMTVmZDMzOTZiNjg0MDkxMjRjZmVlMjlmYzhlZmFjN2I1Yzg0NjYzNGZmOTI0ZTYzNWUwZGMlMjIlN0QlMkMlMjJpdGVtSW5kZXglMjIlM0ExNyU3RCU3RA%3D%3D).

## Semantic retrieval

Besides RAG, which includes natural language generation, a user may only be interested in
the semantic retrieval part.

This can be obtained very similarly to RAG, as shown below:

In [16]:
question = "Where is the IBM lab in Zurich?"

# submit natural-language query on collection
question_query = SemanticQuery(
    question=question,
    project=PROJ_KEY,
    data_source=data_source,

    ## optional params
    retr_k=RETR_K,
    # text_weight=TEXT_WEIGHT,
    # rerank=RERANK,
)
api_output = api.queries.run(question_query)
search_result = SearchResult.from_api_output(api_output, raise_on_error=RAISE)

timing = f"overall={api_output.timings.overall:.2f}  "

for k, v in api_output.timings.tasks["QA"].details.items():
    timing += f"{k}={v:.3f}  "

rich.print(search_result)

print(f"{timing.strip()}")

SearchResult(
    search_result_items=[
        SearchResultItem(
            doc_hash='5dfbd8c115a15fd3396b68409124cfee29fc8efac7b5c846634ff924e635e0dc',
            chunk='DocLayNet: A Large Human-Annotated Dataset for Document-Layout Analysis\nMichele Dolfi IBM 
Research Rueschlikon, Switzerland dol@zurich.ibm.com',
            main_path='main-text.4',
            path_group=['main-text.1', 'main-text.4'],
            source_is_text=True
        )
    ]
)

overall=0.61  pipeline=0.078  encode=0.038  search=0.038  rerank=0.000
